In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from time import sleep

In [2]:
from config import tracker2, bokeh_viz

In [3]:
from tqdm import tqdm_notebook as tqdm

In [4]:
from threading import Lock, Thread

In [5]:
class FrameHelper:
    def __init__(self, frame0):
        self.frame = frame0
        self.h_lock = Lock()
        self.f_lock = Lock()
        self.hands = []

In [6]:
def detector_func(helper):
    i = 0
    while True:
        if i % 10 == 0:
            tracker2.reset()
        new_hands = tracker2(hlp.frame.copy(), hands=None if i % 10 == 0 else hlp.hands)
        hlp.hands = new_hands
        i+=1

In [ ]:
cap = cv2.VideoCapture(0) #("rtsp://192.168.1.162:8083/h264_ulaw.sdp")

frame = cap.read()[1][:,:,::-1]
hlp = FrameHelper(frame)
f = bokeh_viz(frame, scale=0.8)

t_det = Thread(target=detector_func, args=(hlp, ))

t_det.start()

for i in tqdm(range(5000)):
    hlp.frame = cap.read()[1][:,:,::-1]
    # if i % 10 == 0:
    #     tracker2.reset()
    # hands = tracker2(hlp.frame, hands=None if i % 10 == 0 else hands)
#     sleep(0.1)
    frame = hlp.frame.copy()
    for h in hlp.hands:
        jnt = h['joints'].astype('int')

        idx = [0,1,2,5,9,13,17,0]
        for b,e in zip(idx, idx[1:]):
            cv2.line(frame, tuple(jnt[b]),tuple(jnt[e]), (0,255,0), 3)


        for beg in [1,5,9,13,17]:
            idx = [beg+i for i in range(4)]
            for b,e in zip(idx, idx[1:]):
                cv2.line(frame, tuple(jnt[b]),tuple(jnt[e]), (0,255,0), 3)
            
    f(frame)
    
cap.release()

Loading BokehJS ...

/home/wolterlw/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()
